In [1]:
import numpy as np
from matplotlib import pyplot as pl
import os
import scipy.io.wavfile as wav
from python_speech_features import mfcc

%matplotlib inline

In [13]:
PATH = 'vowels/'
MALE = 'm'
FEMALE = 'f'
KID = 'k'

In [28]:
mfccs = []

for filename in os.listdir(PATH):
    if filename.endswith('.wav'):
        sample_rate, X = wav.read(os.path.join(PATH, filename))
        ceps = mfcc(X, samplerate=sample_rate, nfft=1024)
        
        speaker = ''
        if filename[2] == 'm':
            speaker = MALE
        elif filename[2] == 'f':
            speaker = FEMALE
        elif filename[1] == 'k':
            speaker = KID
        else:
            continue
        
        for window in ceps:
            i = 0
            for sample in window:
                mfccs.append((sample, i, speaker))
                i+=1
# mfccs contient désormais toutes les valeurs de mfcc, ainsi que la position du mfcc dans la window et le type de speaker


(-4.611745999023921, 7, 'm')
